In [18]:
%reload_ext autoreload
%autoreload 2

import time

import numpy as np
from ase.io import read
from ase import Atoms

# Imports specific to this "library"
from pylode import DensityProjectionCalculator
from pylode.utilities.generate_invariants import generate_degree2_invariants, generate_degree2_invariants_from_different

## Generate Density Projection Coefficients

In [46]:
# Generate a simple data set containing 5 O2 molecules
frames = []
cell = np.eye(3) *12
distances = np.linspace(1.5, 2., 5)
for d in distances:
    positions = [[1,1,1],[1,1,d+1]]
    frame = Atoms('O2', positions=positions, cell=cell, pbc=True)
    frames.append(frame)
species_dict = {'O':0}

# Define hyperparameters
hypers = {
    'smearing':1.5,
    'max_angular':3,
    'max_radial':4,
    'cutoff_radius':5.,
    'potential_exponent':0,
    'radial_basis': 'gto',
    'compute_gradients':False
    }

# Generate short range (SR) density projection coefficients
# that use a Gaussian density (set potential_exponent = 0 in hypers)
calculator_sr = DensityProjectionCalculator(**hypers)
calculator_sr.transform(frames, species_dict)
descriptors_sr = calculator_sr.features

# Generate long range (LR) density projection coefficients
# that use a smeared Coulomb density (set potential_exponent = 1 in hypers)
hypers['potential_exponent'] = 1
calculator_lr = DensityProjectionCalculator(**hypers)
calculator_lr.transform(frames, species_dict)
descriptors_lr = calculator_lr.features

## Generate invariants from the projection coefficients

In [47]:
# SOAP vectors of the structures: obtained by taking quadratic invariants
# combining SR times SR descriptors (rho x rho)
invariants_soap = generate_degree2_invariants(descriptors_sr)

# Pure LODE invariants: obtained by taking quadratic invariants
# combining LR times LR descriptors (V x V)
invariants_lode = generate_degree2_invariants(descriptors_lr)

# Multiscale invariants: combine SR and LR descriptors (rho x V)
invariants_multiscale = generate_degree2_invariants_from_different(descriptors_sr, descriptors_lr)

## Test invariance

In [49]:
Nrot = 10
np.random.seed(4823)
from numpy.testing import assert_allclose
for i in range(Nrot):
    # Generate random rotation matrix
    from scipy.linalg import qr
    M = np.random.normal(0,1,(3,3))
    Q, R = qr(M)
    assert_allclose(Q.T@Q, np.eye(3), rtol=1e-15, atol=1e-15)
    # Generate rotated structures
    frames_rotated = []
    for d in distances:
        positions = np.array([[1,1,1],[1,1,d+1]]) @ Q.T
        cell_rot = cell @ Q.T
        frame = Atoms('O2', positions=positions, cell=cell_rot, pbc=True)
        frame.wrap()
        frames_rotated.append(frame)
        
    calculator_rot = DensityProjectionCalculator(**hypers)
    calculator_rot.transform(frames_rotated, species_dict)
    descriptors_rot = calculator_rot.features
    invariants_rot = generate_degree2_invariants(descriptors_rot)
    
    diff = invariants_lode - invariants_rot
    error = np.linalg.norm(diff) / np.linalg.norm(invariants_lode)
    print(f'{error:4.2e}')
    #if error > 1e-10: print(diff)

2.09e-15
1.32e-15
6.54e-05
[[[[-4.62542797e-04 -4.61300863e-05  2.11000697e-07 -1.74467834e-06]
   [ 5.31405379e-05 -5.97746633e-05 -1.46887378e-06  1.65611999e-05]
   [-5.22455056e-04 -1.05284507e-04 -4.41846671e-07 -8.40736280e-07]
   [-1.66914698e-04 -3.36553695e-04 -9.37035176e-06  8.26878778e-06]
   [ 2.49187936e-04 -4.11744604e-05 -1.46711606e-05 -1.24869678e-04]
   [ 4.69301541e-04 -2.96665541e-05 -1.36190163e-06  6.18920348e-05]
   [ 1.32071441e-04 -1.38332197e-04 -3.62611720e-05 -1.18053780e-04]
   [ 6.60135034e-05  7.38558571e-05  1.34217754e-07  8.94773889e-05]
   [-7.54521007e-06  1.08086047e-04  1.53109379e-06 -6.19759615e-05]
   [-1.03093004e-05 -1.15097345e-05  2.38947396e-06  9.21600208e-06]]]


 [[[-4.62542797e-04 -4.61300863e-05  2.11000697e-07 -1.74467834e-06]
   [ 5.31405379e-05 -5.97746633e-05 -1.46887378e-06  1.65611999e-05]
   [-5.22455056e-04 -1.05284507e-04 -4.41846671e-07 -8.40736280e-07]
   [-1.66914698e-04 -3.36553695e-04 -9.37035176e-06  8.26878778e-06]
   

1.53e-15
8.89e-16
3.53e-05
[[[[-2.31265162e-04 -1.26383157e-05  1.04765481e-07 -9.37518176e-07]
   [ 2.65649191e-05 -2.52840845e-05 -7.37303618e-07  8.95402508e-06]
   [-2.61227912e-04 -2.35005412e-05 -2.20807351e-07 -5.58159932e-07]
   [-8.34571471e-05 -1.53034693e-04 -4.68508205e-06  5.88126288e-06]
   [ 1.24598557e-04 -1.85660049e-05 -7.34676345e-06 -6.86097987e-05]
   [ 2.34651100e-04 -3.02697265e-06 -6.80498360e-07  3.43596624e-05]
   [ 6.60355473e-05 -6.29270922e-05 -1.81302200e-05 -8.32486081e-05]
   [ 3.30067753e-05  1.67361713e-05  6.70905710e-08  4.95060799e-05]
   [-3.77261743e-06  4.96118984e-05  7.65532086e-07 -4.29876507e-05]
   [-5.15464367e-06 -5.23604729e-06  1.19472500e-06  6.47043405e-06]]]


 [[[-2.31265162e-04 -1.26383157e-05  1.04765481e-07 -9.37518176e-07]
   [ 2.65649191e-05 -2.52840845e-05 -7.37303618e-07  8.95402508e-06]
   [-2.61227912e-04 -2.35005412e-05 -2.20807351e-07 -5.58159932e-07]
   [-8.34571471e-05 -1.53034693e-04 -4.68508205e-06  5.88126288e-06]
   